# Serving PyTorch Models In Production Natively With Amazon SageMaker

# Setup Your Hosting Environment
The focus of this lab is around model serving. In that vain, we have taken care of of the data preparation and model training. 
This lab exercise is using a [HuggingFace Transformer](https://huggingface.co/transformers/) which provides us with a general-purpose architecture for Natural Language Understanding (NLU). Specifically, we are presenting you with a [RoBERTa base](https://huggingface.co/roberta-base) transformer that was fined tuned to perform sentiment analysis. The pre-trained checkpoint loads the additional head layers and will output ``positive``, ``neutral``, and ``negative`` sentiment or text. 

In [ ]:
import boto3
import sagemaker
import pandas as pd

from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch.model import PyTorchModel

from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
%store -r model_s3_uri
print(model_s3_uri)

# Create Your Endpoint
We will now create and deploy our model. To begin, we need to construct a new PyTorchModel object which points to the pre-trained model artifacts from the above step and also points to the inference code that we wish to use. We will then call the deploy method to launch the deployment container on our TorchServe powered Amazon SageMaker endpoint.

In [ ]:
class SentimentAnalysis(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, 
                         sagemaker_session=sagemaker_session, 
                         serializer=JSONSerializer(), 
                         deserializer=JSONDeserializer()
        )

In [ ]:
model_name = name_from_base('roberta-base')
print(model_name)

In [ ]:
model = PyTorchModel(model_data=model_s3_uri,
                     name=model_name,
                     role=role, 
                     entry_point='inference.py',
                     source_dir='code',
                     framework_version='1.6.0',
                     py_version='py3',
                     predictor_cls=SentimentAnalysis)

In [ ]:
endpoint_name = name_from_base('roberta-model') 
print(endpoint_name)

In [ ]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m5.4xlarge', 
                         endpoint_name=endpoint_name, 
                         wait=False)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, endpoint_name)))


In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# _Wait Until the ^^ Endpoint ^^ is Deployed_

# Perform Predictions With A TorchServe Backend Amazon SageMaker Endpoint
Here, we will pass sample strings of text to the endpoint in order to see the sentiment. We give you one example of each, however, feel free to play around and change the strings yourself! 

In [ ]:
# Our endpoint's model should predict a positive sentiment from the text below
test_data = {"review_body": "I love the product"}

In [ ]:
prediction = predictor.predict(test_data)

In [ ]:
print('Review text: {}'.format(test_data))
print('Sentiment: {}'.format(prediction))

In [ ]:
# Our endpoint's model should predict a neutral sentiment from the text below
test_data = {'review_body': 'It is OK.'}

In [ ]:
prediction = predictor.predict(test_data)

In [ ]:
print('Review text: {}'.format(test_data))
print('Sentiment: {}'.format(prediction))

In [ ]:
# Our endpoint's model should predict a negative sentiment from the text below
test_data = {'review_body': 'Worst product ever.'}

In [ ]:
prediction = predictor.predict(test_data)

In [ ]:
print('Review text: {}'.format(test_data))
print('Sentiment: {}'.format(prediction))

## Environment Clean-Up: Delete Endpoint, Endpoint Configuration, and Model
In order to ensure that we are no longer being billed for the endpoint or it's associated resrouces that we have spun up, we use the below steps to tear the environment down. 

In [ ]:
#predictor.delete_endpoint(delete_endpoint_config=True)
#predictor.delete_model()

# Pass Variables to the Next Notebook(s)

In [ ]:
%store model_name

In [ ]:
%store endpoint_name

In [ ]:
%store

# Release Resources

In [ ]:
#%%javascript
#Jupyter.notebook.save_checkpoint();
#Jupyter.notebook.session.delete();